In [27]:
from tensorflow import keras
from tensorflow.keras.layers import Dense, Input, Flatten, concatenate, Lambda, BatchNormalization, Dropout
from tensorflow.keras.models import Model
import pandas as pd
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
import numpy as np
import tensorflow_hub as hub
import cv2
import csv
from tensorflow.keras.datasets import fashion_mnist
import numpy as np
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.utils.vis_utils import plot_model
import tensorflow.keras.backend as K
from tensorflow.keras.applications.resnet50 import ResNet50
import cv2
import cvlib as cv

In [2]:
random_number = 42

In [3]:
tf.keras.utils.set_random_seed(random_number)

In [4]:
import os
os.environ["TFHUB_CACHE_DIR"] = "C:/Users/karel"
hub_extractor = "https://tfhub.dev/google/bit/m-r50x1/1"

In [12]:
input_layer = Input(shape=(224,224,3))

In [13]:
feature_extractor_layer = hub.KerasLayer(
    hub_extractor,
    input_shape=(224, 224, 3),
    trainable=False
)(input_layer)

In [7]:
image_folder = './people'

In [8]:
people = ['DL', 'JP', 'KD', 'MV', 'PZ']

In [15]:
model = tf.keras.Model(inputs=input_layer, outputs = feature_extractor_layer)

In [16]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 keras_layer_1 (KerasLayer)  (None, 2048)              23500352  
                                                                 
Total params: 23,500,352
Trainable params: 0
Non-trainable params: 23,500,352
_________________________________________________________________


In [25]:
def get_face(image):
  img = cv2.imread(image)
  faces, confidences = cv.detect_face(img)
  path = image.replace('.jpg', '_cropped.jpg').replace('.jpeg', '_cropped.jpg')

  if os.path.exists(path):
    return path

  for face in faces:
        (startX,startY) = face[0],face[1]
        (endX,endY) = face[2],face[3]
        cropped_image = img[startY:endY, startX:endX]
        cv2.imwrite(path, cropped_image)
        return path

In [31]:
def preprocess_image(image):
    image_string = tf.io.read_file(image)
    image = tf.image.decode_jpeg(image_string)

    resize = tf.keras.layers.Resizing (224, 224, interpolation='bilinear', crop_to_aspect_ratio=False)
    resized_image = resize(image)
    
    resized_image = tf.keras.applications.resnet50.preprocess_input(resized_image)
    resized_image = tf.image.convert_image_dtype(resized_image, tf.float32)

    return resized_image

In [45]:
team_vectors = []

In [46]:
for person in people:
    path = image_folder + '/IMG_' + person + '.jpg'
    path = get_face(path)
    
    img = preprocess_image(path)
    img = np.expand_dims(img, axis=0)
    
    pred = model.predict(
        img,
        batch_size = 1,
        verbose=0
    )
    
    team_vectors.append([person, pred[0].tolist()])

In [48]:
import pandas as pd
import numpy as np
pd.DataFrame(team_vectors).to_csv('team_vectors.csv')  